# Youtube spam detection

Download [link](http://www.dt.fee.unicamp.br/~tiago//youtubespamcollection/)

In [1]:
from fastai2.text.all import *
from lazylabel.text.all import *

In [2]:
source = Path('/home/lgvaz/.data/youtube_spam/')

In [3]:
fns = get_files(source, extensions='.csv')
dfs = [pd.read_csv(fn) for fn in fns]

In [4]:
df = pd.concat(dfs)
print(df.size); df.head()

9780


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z12pgdhovmrktzm3i23es5d5junftft3f,lekanaVEVO1,2014-07-22T15:27:50,i love this so much. AND also I Generate Free Leads on Auto Pilot &amp; You Can Too! http://www.MyLeaderGate.com/moretraffic﻿,1
1,z13yx345uxepetggz04ci5rjcxeohzlrtf4,Pyunghee,2014-07-27T01:57:16,http://www.billboard.com/articles/columns/pop-shop/6174122/fan-army-face-off-round-3 Vote for SONES please....we're against vips....please help us.. &gt;.&lt;﻿,1
2,z12lsjvi3wa5x1vwh04cibeaqnzrevxajw00k,Erica Ross,2014-07-27T02:51:43,"Hey guys! Please join me in my fight to help abused/mistreated animals! All fund will go to helping pay for vet bills/and or helping them find homes! I will place an extra emphasis on helping disabled animals, ones otherwise would just be put to sleep by other animal organizations. Donate please. http://www.gofundme.com/Angels-n-Wingz﻿",1
3,z13jcjuovxbwfr0ge04cev2ipsjdfdurwck,Aviel Haimov,2014-08-01T12:27:48,http://psnboss.com/?ref=2tGgp3pV6L this is the song﻿,1
4,z13qybua2yfydzxzj04cgfpqdt2syfx53ms0k,John Bello,2014-08-01T21:04:03,Hey everyone. Watch this trailer!!!!!!!! http://believemefilm.com?hlr=h2hQBUVB﻿,1


In [5]:
df['original'] = df['CONTENT']

In [6]:
cder,tkzer,nmzer = mk_transform(ColReader('original')),Tokenizer.from_df('CONTENT'),Numericalize()

In [7]:
ABSTAIN,SPAM,HAM = 'abstain','spam','ham'
vocab = [ABSTAIN,SPAM,HAM]

In [8]:
labeller = Labeller()
@labeller(tkzer)
def check(x): return SPAM if 'check' in x else ABSTAIN
@labeller(cder)
def check_out(x): return SPAM if 'check out' in x.lower() else ABSTAIN

In [9]:
tls = TfmdLists(df, [cder,tkzer,nmzer])
tasks = tasks_labels(labeller, tls, vocab)

In [10]:
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [11]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.63% (396),1
check,24.11% (463),1


In [12]:
def get_sentences(tls, idxs):
    sentences = {}
    for i in idxs:
        tks = tls.decode(tls[i])
        sentences[i] = (' '.join(decode_spec_tokens(tks.split(' '))))
    return pd.DataFrame(sentences.values(), sentences.keys())

In [13]:
idxs = labeller.find(dls.train, vocab, ['check', 'check_out'], [SPAM, ABSTAIN])

In [14]:
get_sentences(tls, idxs).sample(10)

,0
159,"xxbos a friend of mine has xxunk a big dick xxunk . He had a small dick ( 4 xxunk ) and he did some xxunk about this xxunk . Xxunk the xxunk , he found out the secret xxunk of xxunk xxunk . He xxunk what he had learned and now he has a 7 xxunk dick . He was absolutely xxunk by his xxunk . Of xxunk , it took a few months . Xxunk , he has xxunk a book about this xxunk in order to help guys . He xxunk me to xxunk his book :) So , guys if you are xxunk and for more xxunk check this out https : / / xxunk / b / xxunk . He is xxunk it for xxunk xxunk 1 . Thank you ; ) ﻿"
801,xxbos Xxunk it you just came here to check the number of xxunk ﻿
776,xxbos hi guys check my youtube channel﻿
91,xxbos Can you check my videos please ? Do n't hate me :( Give me one xxunk
708,"xxbos Hey Music Fans i really appreciate all of you who take time to read this , and check my music out ! I&'m just a 15 year old boy DREAMING of being a successful MUSICIAN in the music world . i do lots of covers , and piano covers . But i do nt have money to advertise . a simple thumbs up to my comment , a comment on my videos or a SUBSCRIPTION would be a step forward ! It will only be a few seconds of your life that u won&'t regret !!! Thank u to all the people who just give me a chance it means a lot ! :)"
294,xxbos Such a good song xxunk Katy sounds great over the melody . I m xxunk as an artist everyday from my hit song ' crazy ' which has got my name out there . ca nt thank my fans more for their support . If you could take a moment ? to check it and my music ? maybe you 'll join me to make my dream come true :) thank you for your time﻿
724,xxbos watch?v = xxunk Check this out .﻿
10,xxbos Hey Guys this is Xxunk Beauty ! i just started my youtube channel please go check it out ! I 'm going to post singing videos and also xxunk videos that i xxunk like ! Please go to subscribe ! More to come xxunk Remember to subscribe!﻿
861,xxbos check men out i put xxunk of xxunk into my music but xxunk not many watch it﻿
1033,xxbos Hey guys can you check my channel out plz . i do mine xxunk videos . Let 's xxunk for 20 xxunk


In [15]:
@labeller(cder)
def regex_check_out(x):
    return SPAM if re.search(r'check.*out', x, flags=re.I) else ABSTAIN

In [16]:
@labeller(cder)
def regex_check(x):
    return SPAM if re.search(r'(check\s(my|this))|check\s(\w+)?\s?out', x, flags=re.I) else ABSTAIN

In [17]:
tasks = tasks_labels(labeller, tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [18]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.42% (392),1
regex_check_out,22.45% (431),1
regex_check,23.07% (443),1
check,23.91% (459),1


In [19]:
idxs = labeller.find(dls.train, vocab, ['regex_check', 'regex_check_out'], [SPAM, ABSTAIN])
print(len(idxs))
get_sentences(tls, idxs).sample(10)

17


,0
880,xxbos CHECK MY CHANNEL FOR MY NEW SONG ' xxunk ' ! ! YOU 'll LOVE XXUNK
224,xxbos want to win xxunk the xxunk - xxunk ? check my channel : ) ﻿
209,"xxbos Rap from Xxunk , check my xxunk"
1329,xxbos ┏ ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ┏┏┏ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ╋╋╋ ┏┏┏ ┓┓┓ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ ┃┃┃ ┃┃┃ ╋╋╋ 444 ┃┃┃ ┗┗┗ ┓┓┓ ┗┗┗ ┛┛┛ ┏┏┏ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ ┃┃┃ ╋╋╋ 444 ┃┃┃ ╋╋╋ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┏┏┏ ┛┛┛ ┗┗┗ ┛┛┛ ┃┃┃ ╋╋╋ ╋╋╋ ┃┃┃ ┃┃┃ ┗┗┗ 333 ━━━ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ 333 ━━━ ┛┛┛ ╋╋╋ ╋╋╋ ┗┗┗ ┛┛┛ CHECK MY VIDEOS AND SUBSCRIBE AND LIKE XXUNK
764,"xxbos if you like xxunk talent , xxunk lyrics , xxunk real hip hop Everyone check my newest sound Xxunk x - Got the Xxunk ( prod by . Xxunk the Model Xxunk ) COMMENT TELL ME WHAT YOU THINK DO nt BE XXUNK !!!! ---- 1111 //// 7777 Xxunk"
940,"xxbos I m a RAPPER / SONGWRITER , check my video please .. also subscribe for more thanks :) tell me what you xxunk"
729,xxbos and u xxunk check my channel and tell me what i should do xxunk
25,xxbos Hi Guys ! check this awesome EDM & & House mix :) thanks a lot .. https : / / soundcloud.com / xxunk / edm - house - xxunk
756,xxbos Check my channel please ! And listen to the best music ever : p﻿
812,"xxbos Check my channel , xxunk"


In [20]:
idxs = labeller.find(dls.train, vocab, ['regex_check'], [SPAM])
get_sentences(tls, idxs).sample(10)

,0
756,xxbos Check my channel please ! And listen to the best music ever : p﻿
1120,"xxbos Xxunk xxunk check out our remix to 50 Xxunk Your Life Is On The Xxunk we just started our youtube channel and we are all xxunk working hard , give us some feed back on our latest song on what you guys think if you like show xxunk"
317,"xxbos Hey Guys , i know you xxunk to skip these comments , but Take a look first . i am Free Fire , a Big Xxunk , Xxunk House , Xxunk House , Dubstep & & Xxunk Producer from a small town in Xxunk . Down here , i have no support from the xxunk , and all i really ask is some xxunk , Support , and Views . My Music has xxunk much better then what it was 10 months ago , and i promise my new content should be more then xxunk . Soon i should be xxunk to advertise and i wo nt xxunk you guys anymore . So xxunk a chance and check out my music , thanks and god xxunk"
1378,xxbos could you guys please check out my channel for xxunk beats ?
1328,xxbos i can&'t believe this i just now watched a brand new xxunk video clip with rihanna She has been xxunk a black Xxunk player Check out her video right here if you wish : xxunk
64,xxbos GO TO MY CHANNEL and check out my xxunk songs﻿
1809,xxbos Check out this video on youtube : xxunk mm \n\n\n\n xxunk xxunk xxunkxxunkxxunkxxunk 5555 ==== ++++ ++++ xxunkxxunkxxunkxxunk xxunkxxunkxxunkxxunk 3333 xxunkxxunkxxunkxxunk \n\n\n\n ﻿﻿﻿﻿
1895,xxbos Hi everyone ! Do you like music ? Then why not check out my music channel . The LEXIS band will be uploading their own songs and covers soon so don&'t miss out . Please SUBSCRIBE too as it does help us out a lot . Just takes one click . ->﻿
1376,xxbos CHECK OUT MY MUSIC VIDEO ON MY CHANEL !!!
206,"xxbos Anyone Who loves music , please go check out my youtube page and tell me what you think . i just put a video up and will be doing more song . I 'm just trying to get myself started . Any love is much Appreciated ﻿"


In [22]:
labeller.remove('check_out')
labeller.remove('regex_check_out')

In [23]:
from textblob import TextBlob

In [24]:
@labeller(cder, TextBlob)
def tb_polarity(x, tb):
    return HAM if tb.polarity > 0.9 else ABSTAIN
@labeller(cder, TextBlob)
def tb_sentiment(x, tb):
    return HAM if tb.subjectivity >= 0.5 else ABSTAIN

In [25]:
tasks = tasks_labels(labeller, tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [26]:
labeller.summary(dls.train)

,Coverage,Polarity
regex_check,23.13% (444),1
tb_polarity,4.79% (92),1
tb_sentiment,38.13% (732),1
check,23.91% (459),1


In [27]:
idxs = labeller.find(dls.train, vocab, lfs=['tb_sentiment'], lbls=[HAM])
get_sentences(tls, idxs).sample(10)

,0
937,xxbos 2 Billion Views For This Piece Of Shit … ~ a r e ~ y o u ~ xxunk i d d i n g ~ m e xxunk
54,xxbos XXUNK I m THE xxunk XXUNK !!!!! IF YOUR CRAZY PLEASE DONATE XXUNK TO ME AT : xxunk
383,xxbos BEAUTIFUL
149,xxbos my son love so much﻿
982,"xxbos THIS HAS MORE VIEWS THAN QUEEN AND MICHAEL JACKSON , 2 BILLION views omg﻿"
866,xxbos Get free gift cards and pay xxunk xxunk
717,xxbos wow
1514,xxbos Check out my videos guy ! :) Hope you guys had a good laugh : d
37,xxbos http : / / xxunk / xxunk / xxunk / = ) ﻿
422,xxbos i swear Shakira xxunk getting more and more gorgeous ! She xxunk looks more gorgeous with her hair this way than super xxunk


In [28]:
def keyword_match(ws, label=SPAM):
    ws = L(ws)
    def _inner(x):
        for w in ws:
            if w in x: return label
        return ABSTAIN
    _inner.__name__ = f'kw_{ws[0]}'
    labeller.register_func(cder, _inner)

In [29]:
keyword_match('my')
keyword_match('subscribe')
keyword_match(['http', 'https'])
keyword_match(['please', 'plz', 'pliz'])
keyword_match(['song'], label=HAM)

In [30]:
@labeller(tkzer)
def short_comment(x): return HAM if len(x) < 6 else ABSTAIN

In [31]:
tasks = tasks_labels(labeller, tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [33]:
labeller.summary(dls.train)

,Coverage,Polarity
regex_check,23.49% (451),1
tb_polarity,4.64% (89),1
tb_sentiment,37.92% (728),1
kw_my,16.46% (316),1
kw_subscribe,8.28% (159),1
kw_http,9.90% (190),1
kw_please,6.15% (118),1
kw_song,14.38% (276),1
check,24.32% (467),1
short_comment,11.77% (226),1
